In [ ]:
!pip install cloudscraper

In [ ]:
import cloudscraper
from bs4 import BeautifulSoup

url = "https://vinpearl.com/vi/top-20-quan-cafe-dep-o-sai-gon-nen-ghe-du-chi-mot-lan"
scraper = cloudscraper.create_scraper()
rq = scraper.get(url)
doc = BeautifulSoup(rq.text)
doc = doc.find(class_="content")

In [ ]:
place = []
for item in doc.find_all('h3'):
  place.append(item.get_text())

In [ ]:
address = []
time = []
ul_tag = doc.find_all('ul')[:-2]
index_without_address_time = [12, 13]

i = -1
for item in ul_tag:
  i = i + 1
  if (i in index_without_address_time):
    address.append("No Information From Website")
    time.append("No Information From Website")
    address.append("No Information From Website")
    time.append("No Information From Website")
    li_tag = item.find_all('li')
    address.append(li_tag[0].get_text().split(": ")[1])
    time.append(li_tag[1].get_text().split("Giờ mở cửa:")[1][1:])
    i = 100
  else:
    li_tag = item.find_all('li')
    address.append(li_tag[0].get_text().split(": ")[1])
    time.append(li_tag[1].get_text().split("Giờ mở cửa:")[1][1:])

In [ ]:
img_src = []
index_with_3_cap = [12, 13]
index_with_2_cap = [19]
index_with_both_type = [15, 16]
index_with_3_uncap = [4]
index_with_2_uncap = [17]

cap = [item for item in doc.find_all('img', {"alt":"quán cafe đẹp ở Sài Gòn"})]
uncap = [item for item in doc.find_all('img', {"alt":"quán cafe đẹp ở sài gòn"})]

for i in range(20):
  if i in index_with_3_cap:
    group = [cap.pop(0), cap.pop(0), cap.pop(0)]
  elif i in index_with_2_cap:
    group = [cap.pop(0), cap.pop(0)]
  elif i in index_with_both_type:
    group = [cap.pop(0), uncap.pop(0)]
  elif i in index_with_3_uncap:
    group = [uncap.pop(0), uncap.pop(0), uncap.pop(0)]
  elif i in index_with_2_uncap:
    group = [uncap.pop(0), uncap.pop(0)]
  else:
    group = [cap.pop(0), uncap.pop(0), uncap.pop(0)]
  img_src.append(group)

img_src_copy = img_src.copy()
img_src.clear()
for item in img_src_copy:
  temp = []
  for each in item:
    temp.append(each.get("src"))
  img_src.append(temp)

In [ ]:
p_tag = []
for item in doc.find_all('p')[1:-10]:
  if ("42 Thảo Điền" not in item.get_text() and
      item.find('a') == None or
      "chợ Bến Thành" in item.get_text() or
      "phố đi bộ Nguyễn Huệ" in item.get_text() or
      "Dinh Độc Lập" in item.get_text() or
      "phố Tây Bùi Viện" in item.get_text()):
    p_tag.append(item)

description = []
index_with_3_des = [4, 12, 13]
index_with_1_des = [15, 19]
for i in range(20):
  if i in index_with_3_des:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text())
  elif i in index_with_1_des:
    description.append(p_tag.pop(0).get_text())
  else:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text())

In [ ]:
data = []
for i in range(20):
  data_itr = dict()
  data_itr['Place'] = place[i]
  data_itr['Address'] = address[i]
  data_itr['Time'] = time[i]
  data_itr['Image Source'] = img_src[i]
  data_itr['Description'] = description[i]

  data.append(data_itr)

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.to_csv("CrawlData_vinpearl_cafe.csv", encoding="utf-8-sig", index = False)